In [1]:
!pip install  --quiet kfp==1.8.22 

In [2]:
# Restart the kernel to have the new kfp version installed

In [3]:
import kfp
from kfp import dsl
import kfp.components as components
# from kfp.components import InputPath, InputTextFile, OutputPath, OutputTextFile

from typing import NamedTuple

In [4]:
%run 020-pipeline-step-01.ipynb
%run 020-pipeline-step-02.ipynb
%run 020-pipeline-step-03.ipynb

Overwriting py_src/mnist_model.py


In [5]:

import kfp
from kfp import dsl
import kfp.components as components

step_put_public_mnist_to_local_minio = components.create_component_from_func(put_public_mnist_to_local_minio,base_image="kubeflownotebookswg/jupyter-pytorch-full")
step_train_model = components.create_component_from_func(train_model,base_image="kubeflownotebookswg/jupyter-pytorch-full")
step_test_model_and_save_metrics = components.create_component_from_func(test_model_and_save_metrics,base_image="kubeflownotebookswg/jupyter-pytorch-full")

@dsl.pipeline(
    name='kfp-pipeline-digits-clsfier',
    description='Classify digits'
)
def kfp_pipeline_start(args:dict ):
    # pipeline_arguments = arg_wrapper["pipeline_arguments"]

    step1 = step_put_public_mnist_to_local_minio(args)
    step2 = step_train_model(args)
    step3 = step_test_model_and_save_metrics(args)

    step2.after(step1)
    step3.after(step2)






In [6]:
import os
host, port = os.environ["ML_PIPELINE_PORT_8888_TCP_ADDR"],os.environ["ML_PIPELINE_SERVICE_PORT_HTTP"]
kfp_endpoint="http://{}:{}".format(host,port)

kfp_client = kfp.Client(host=kfp_endpoint)

pipeline_arguments = {
    "args" : {
        "epochs": 1,
        "optimizer": "adam",
        "batch_size":64,
        "test_batch_size":1000,
        "device_name":"cpu",
        "epochs":1,
        "lr":0.03,
        "gamma":0.7,
        "seed":1,
        "log_interval":100,
        "model_save_prefix": "models/trained/detect-digits",
        "bucket_name":"datapipeline-031",
        "take_nth_in_subset":10,
        "version": "22"
    }
}
pipeline_run = kfp_client.create_run_from_pipeline_func(kfp_pipeline_start, arguments=pipeline_arguments)


In [7]:
import os
from IPython.display import IFrame
cluster_id= os.environ["MEDDASH_CLUSTER_ID"]
run_id=pipeline_run.run_id
print("pipeline.run_id:", run_id)
pipeline_vis_endpoint="http://kf-ml-pipeline-ui.{}.hosted.meddash.cloud/#/runs/details/{}".format(cluster_id,run_id)
IFrame(pipeline_vis_endpoint, width=1200, height=600)

pipeline.run_id: 7f1f7dc5-177a-4c40-a56a-27d90f9380fd
